In [1]:
# import the necessary packages
from jinja2 import Environment
from jinja2 import FileSystemLoader
from tqdm import tqdm
import pandas as pd
import openai
import json
import time
import os

In [3]:
# set your OpenAI API key
# openai.api_key = ""

In [4]:
# specify the path to the EM-DAT dataset
emdat_dataset_path = os.path.join(
    "natural-disasters-data",
    "em-dat",
    "EMDAT_1900-2021_NatDis.csv"
)

# load the EM-DAT natural disasters dataset from disk
df = pd.read_csv(emdat_dataset_path)
df.head()

,Dis No,Year,Seq,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Disaster Subsubtype,Event Name,Entry Criteria,...,End Day,Total Deaths,No Injured,No Affected,No Homeless,Total Affected,Reconstruction Costs ('000 US$),Insured Damages ('000 US$),Total Damages ('000 US$),CPI
0,1900-9002-CPV,1900,9002,Natural,Climatological,Drought,Drought,NaN,NaN,NaN,...,NaN,11000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.261389
1,1900-9001-IND,1900,9001,Natural,Climatological,Drought,Drought,NaN,NaN,NaN,...,NaN,1250000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.261389
2,1902-0012-GTM,1902,12,Natural,Geophysical,Earthquake,Ground movement,NaN,NaN,Kill,...,18.0,2000.0,NaN,NaN,NaN,NaN,NaN,NaN,25000.0,3.391845
3,1902-0003-GTM,1902,3,Natural,Geophysical,Volcanic activity,Ash fall,NaN,Santa Maria,Kill,...,8.0,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.391845
4,1902-0010-GTM,1902,10,Natural,Geophysical,Volcanic activity,Ash fall,NaN,Santa Maria,Kill,...,24.0,6000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.391845


In [6]:
# grab all tornadoes that have occurred in the United States
tornado_us_df = df[
    (df["Disaster Subsubtype"] == "Tornado") &
    (df["ISO"] == "USA")
]
tornado_us_df.head()

,Dis No,Year,Seq,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Disaster Subsubtype,Event Name,Entry Criteria,...,End Day,Total Deaths,No Injured,No Affected,No Homeless,Total Affected,Reconstruction Costs ('000 US$),Insured Damages ('000 US$),Total Damages ('000 US$),CPI
873,1903-0010-USA,1903,10,Natural,Meteorological,Storm,Convective storm,Tornado,NaN,Kill,...,NaN,98.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.522300
908,1913-0005-USA,1913,5,Natural,Meteorological,Storm,Convective storm,Tornado,NaN,Kill,...,NaN,732.0,NaN,NaN,NaN,NaN,NaN,NaN,200000.0,3.874530
925,1920-0016-USA,1920,16,Natural,Meteorological,Storm,Convective storm,Tornado,NaN,Kill,...,20.0,224.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.853425
939,1925-0001-USA,1925,1,Natural,Meteorological,Storm,Convective storm,Tornado,NaN,Kill,...,18.0,739.0,NaN,NaN,NaN,NaN,NaN,NaN,18000.0,6.875008
949,1927-0010-USA,1927,10,Natural,Meteorological,Storm,Convective storm,Tornado,NaN,Kill,...,NaN,87.0,NaN,NaN,NaN,NaN,NaN,NaN,22000.0,6.809780


In [7]:
# examine the locations of the tornadoes in the United States
tornado_us_df["Location"]

873                                  Gainesville (Georgia)
908                                   Ohio, Indiana, Texas
925                                   Alabama, Mississippi
939                             Montana, Indiana, Illinois
949                                      Tornado, St-Louis
                               ...                        
15298    Alabama, Georgia, South Carolina, Florida, Mis...
15780    Illinois, Iowa, Wisconsin, Michigan, Indiana, ...
15785    Louisiana, Texas, Mississippi, South Carolina,...
15786    Texas, Oklahoma, Louisiana, Mississippi, Alaba...
15798    Nashville (Tennessee), Kentucky, Missouri, Mis...
Name: Location, Length: 144, dtype: object

In [8]:
# to fully appreciate why the 'location' column is so problematic (and in need
# of AI intervention, otherwse we'd be forced to *manully* edit them), let's
# look at *all* the values (notice how there are typos in the state names,
# other times the states aren't listed and instead towns, cities, and/or
# counties are intermingled, etc.)
print("\n".join(tornado_us_df["Location"]))

Gainesville (Georgia)
Ohio, Indiana, Texas
Alabama, Mississippi
Montana, Indiana, Illinois
Tornado, St-Louis
Texas, Alabama, South Carolina
Mississippi, Georgia
Massachusetts
Gainesville (Georgie)
Illinois, Indiana, Michigan
Kansas
Lubbock (Texas)
Central
Texas, Oklahoma
Texas, Oklahoma
Texas, Middle West
East Coast
East, South
Colorado
Pennsylvania, Ohio, Missouri, New York
Saragosa (Texas)
Texas, Oklahoma, Louisiane, Mississippi
Huntsville (Alabama)
Texas, Kansas, Iowa, Louisiane, Nebraska, Missouri, Mississippi, Oklahoma, Alabama, Georgie, Arkansas
Nevada, Colorado, Kansas, Nebraska, Oklahoma, Texas
South Dakota, Minnesota
Colorado, Texas, Kansas, Nebraska, Oklahoma, Missouri
Wisconsin, Ohio, Maryland, Michigan, Indiana, Kentucky, Virginie
Kansas, Missouri, Illinois
Washington, Oregon, Texas, Arkansas, Ten Nessee, Illinois, Louisiane, Mississippi
Tulsa, Catoosa (Oklahoma)
Kentucky, Tennessee, Georgia
Georgia, Florida
Oklahoma, Wisconsin, Illinois, Indiana, Ohio, Pennsylvania, New Je

In [11]:
!cat templates/state_mapping_prompt.txt

[Return only the main response. Omit pre-text, post-text, and any explanation]
[Format your response as a JSON variable using the {Output Template}]

Act as both (1) an expert meteorologist and (2) an expert data scientist.

Below you'll find a comma separated list of {locations} that a given tornado was recorded in. These locations could consist of state names, cities, counties, townships, or regions. 

The {locations} data is imperfect, and there may be typos. Do your best to infer the correct names of the locations.

Your job is to take these locations and fill the {output template} JSON variable with the unique names of each state the comma-separated list of locations maps to. When naming the states, use their two letter codes rather than the full name.

In some cases, you may be unable to determine the name of the state from the input. In that case, simply ignore the string that could not be converted into a proper state name.

Locations:
{{ locations }}

Output Template (JSON var

In [9]:
def extract_state_names(
    text,
    template_path="templates/",
    prompt_filename="state_mapping_prompt.txt"
):
    # initialize the list of states
    states = None

    # try to make a request to OpenAI and parse the result
    try:
        # load the prompt template from disk, then render it
        environment = Environment(loader=FileSystemLoader(template_path))
        prompt_template = environment.get_template(prompt_filename)
        prompt = prompt_template.render({"locations": text})
    
        # submit the prompt to OpenAI and obtain the response
        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature=0,
            messages=[{
                "role": "user",
                "content": prompt
            }]
        )
        response = completion.choices[0].message.content
    
        # extract the comma separated list of states from the JSON blob
        states = json.loads(response)["states"]

    # handle if the OpenAI request fails, or if the returned JSON blob cannot
    # be parsed
    except (json.JSONDecodeError, openai.error.APIError) as e:
        print("* error: {}".format(e))

    # return the set of states
    return states

In [10]:
# initialize the list of parsed locations
locations = []

# loop over all rows of the tornado dataframe
for loc in tqdm(tornado_us_df["Location"]):
    # update the list of locations using our OpenAI state parsing function,
    # then include a small sleep to ensure rate limits aren't hit
    locations.append(extract_state_names(loc))
    time.sleep(0.1)

100%|█████████████████████████████████████████████████████████████| 144/144 [03:34<00:00,  1.49s/it]


In [22]:
# build a dataframe that shows our original locations (a mix of states, cities,
# counties, towns, regions, etc.), along with the same location data that has
# been cleaned to include *just* state data
cleaned_df = pd.DataFrame(
    list(zip(tornado_us_df["Location"], locations)),
    columns=["Original Locations", "Locations Cleaned with AI"]
)
cleaned_df.iloc[51:75]

,Original Locations,Locations Cleaned with AI
51,"Osceola, Seminole, Volusia counties, Daytona b...",[FL]
52,"Oklahoma, Kansas","[OK, KS]"
53,"Georgia, Louisiana, Minnesota, Alabama, Florid...","[GA, LA, MN, AL, FL, SC, NC]"
54,"Louisiana, Kansas, Arkansas, Montana, Oklahoma...","[LA, KS, AR, MT, OK, TN, TX]"
55,"Tennessee, Arkansas, Indiana, Alabama, Texas, ...","[TN, AR, IN, AL, TX, MT, LA, MO, GA]"
56,Central Georgia,[GA]
57,"Oklahoma, Kansas, Nebraska, Texas, Arkansas, L...","[OK, KS, NE, TX, AR, LA, MO, AL, FL, GA, IL, I..."
58,Salt Lake City (Utah,[UT]
59,Tennessee,[TN]
60,"Crittenden, Daviess, Webster districts (Kentuc...",[KY]
